In [1]:
import os
import json
import glob
import torch
import argparse
from PIL import Image
from torchvision import transforms as T
from net import get_model
import numpy as np
import time

In [2]:
import time

start = time.time()
time.sleep(5)
stop = time.time()
t = stop-start
t

5.004802465438843

In [10]:
import os
import json
import glob
import torch
import argparse
from PIL import Image
from torchvision import transforms as T
from net import get_model
import numpy as np
import time
def xnor(lst1, lst2):
    arr_result = []
    for count, i in enumerate(lst1):
        if i == lst2[count] == 0:
            arr_result.append(None)
        elif i == lst2[count] == 1:
            arr_result.append(1)
        elif i != lst2[count]:
            arr_result.append(0)
    return arr_result

num_label = 20

transforms = T.Compose([
    T.Resize(size=(288, 144)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

use_id = False
model_name = '{}_nfc_id'.format('resnet50') if use_id else '{}_nfc'.format('resnet50')
# num_label, num_id = num_cls_dict[args.dataset], num_ids_dict[args.dataset]
# print(num_label, num_id)
num_label, num_id = (20,1)

# def load_network(network):
#     save_path = os.path.join('/home/minh/Documents/minh/mqsolutions/detec_color/Person-Attribute-Recognition-MarketDuke/checkpoints/market/resnet50_nfc', 'net_30.pth')
#     network.load_state_dict(torch.load(save_path, map_location="cuda:0"))
#     print('Resume model from {}'.format(save_path))
#     return network

def load_network(network):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    save_path = os.path.join('/home/minh/Documents/minh/mqsolutions/detec_color/Person-Attribute-Recognition-MarketDuke/checkpoints/market/resnet50_nfc', 'net_30.pth')
    network.load_state_dict(torch.load(save_path, map_location=device)) 
    network.to(device)
    print('Resume model from {}'.format(save_path))
    return network

def load_image(path):
    src = Image.open(path)
    src = transforms(src)
    src = src.unsqueeze(dim=0)
    return src.cuda()

model = get_model(model_name, num_label, use_id=False, num_id=num_id)
# model = load_network(model)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
save_path = os.path.join('./checkpoints/market/resnet50_nfc', 'net_last.pth')
model.load_state_dict(torch.load(save_path, map_location=device)) 
model.to(device)
model.eval()

num = 0


with open('./doc/final_label.json', 'r', encoding = 'utf-8') as labels:
    label = json.load(labels)
    label= list(label.values())

path_json = './doc/label_train.json'
with open(path_json) as f:
    label_train = json.load(f)

    

path = './dataset/data_set_minh/train/'

list_dir = os.listdir(path)
print(list_dir)
result = np.empty((0,20), int)
name_imgs = []
k = 0
start = time.time()
for name_folder in list_dir:
    path_folder = path + name_folder +'/'
    for name in glob.glob(path_folder+'*.jpg'):
        ids = name.split('/')[-1].split('.')[0]
        
        image_path = name
        ids=name.split('/')[-1].split('.')[0]
        src = load_image(image_path)
        out = model.forward(src)
        pred = torch.gt(out, torch.ones_like(out)/2 )  # threshold=0.5
        pred = pred.tolist()[0]

        for inter, i in enumerate(pred):
            if i:
                pred[inter] = 1
            else:
                pred[inter] = 0
#         print(pred)
#         print(label_train[ids])
#         print('------------------')
#         print(type(pred))
        compare = xnor(label_train[ids],  pred)
        compare = np.array(compare)
        result = np.vstack((result, compare))  
        k+=1
        if k%250==0:
            stop = time.time()
            t = stop-start
            print(f'{k} :  {t}')
            start = time.time()
            break
        
    
# np.save("result_net_40.npy", result)
a = 0
for count, i in enumerate(label):
    check = result[:, count]
    number = 0
    total = 0
    for j in check:
        if j == 1:
            number += 1
            total += 1
        elif j == 0:
            total += 1
    try:
        print(i, round(number/total, 4))
        a = a + round(number/total, 4)
    except Exception as e:
        print("An exception occurred: ", e) 
        print(i)
print(' acc = ', a/19)

['taxi', 'car', 'bus', 'lorry', 'truck', 'coach']
250 :  27.83106231689453
500 :  24.930705070495605
750 :  33.324243783950806
1000 :  24.80368399620056
1250 :  29.795875549316406
1500 :  29.99821424484253
red 0.8071
orange 0.8083
yellow 0.8685
green 0.7963
blue 0.8539
An exception occurred:  division by zero
purple
brown 0.0
pink 0.0
gray 0.5065
white 0.6512
black 0.6414
An exception occurred:  division by zero
Mini
Medium 0.968
Large 0.9664
bus 0.9923
truck 0.0
car 0.3984
coach 0.98
taxi 0.6335
lorry 0.992
 acc =  0.6244105263157895


In [8]:
a = 0
for count, i in enumerate(label):
    check = result[:, count]
    number = 0
    total = 0
    for j in check:
        if j == 1:
            number += 1
            total += 1
        elif j == 0:
            total += 1
    try:
        print(i, round(number/total, 4))
        a = a + round(number/total, 4)
    except Exception as e:
        print("An exception occurred: ", e) 
        print(i)
print(' acc = ', a/19)

red 0.9763
orange 0.7456
yellow 0.8897
green 0.9202
blue 0.9061
purple 0.0
brown 0.076
pink 0.4716
gray 0.7608
white 0.9535
black 0.8777
An exception occurred:  division by zero
Mini
Medium 0.9952
Large 0.989
bus 0.999
truck 0.198
car 0.9948
coach 0.998
taxi 0.872
lorry 0.9876
 acc =  0.7690052631578947


In [ ]:
a/19

# evalue random

In [1]:
import os
import json
import glob
import torch
import argparse
from PIL import Image
from torchvision import transforms as T
from net import get_model
import numpy as np
import time
def xnor(lst1, lst2):
    arr_result = []
    for count, i in enumerate(lst1):
        if i == lst2[count] == 0:
            arr_result.append(None)
        elif i == lst2[count] == 1:
            arr_result.append(1)
        elif i != lst2[count]:
            arr_result.append(0)
    return arr_result

num_label = 20

transforms = T.Compose([
    T.Resize(size=(288, 144)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

use_id = False
model_name = '{}_nfc_id'.format('resnet50') if use_id else '{}_nfc'.format('resnet50')
# num_label, num_id = num_cls_dict[args.dataset], num_ids_dict[args.dataset]
# print(num_label, num_id)
num_label, num_id = (20,1)

# def load_network(network):
#     save_path = os.path.join('/home/minh/Documents/minh/mqsolutions/detec_color/Person-Attribute-Recognition-MarketDuke/checkpoints/market/resnet50_nfc', 'net_30.pth')
#     network.load_state_dict(torch.load(save_path, map_location="cuda:0"))
#     print('Resume model from {}'.format(save_path))
#     return network

# def load_network(network):
#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     save_path = os.path.join('/home/minh/Documents/minh/mqsolutions/detec_color/Person-Attribute-Recognition-MarketDuke/checkpoints/market/resnet50_nfc', 'net_30.pth')
#     network.load_state_dict(torch.load(save_path, map_location=device)) 
#     network.to(device)
#     print('Resume model from {}'.format(save_path))
#     return network

def load_image(path):
    src = Image.open(path)
    src = transforms(src)
    src = src.unsqueeze(dim=0)
    return src.cuda()

model = get_model(model_name, num_label, use_id=False, num_id=num_id)
# model = load_network(model)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
save_path = os.path.join('./checkpoints/market/resnet50_nfc', 'net_4.pth')
model.load_state_dict(torch.load(save_path, map_location=device)) 
model.to(device)
model.eval()

num = 0


with open('./doc/final_label.json', 'r', encoding = 'utf-8') as labels:
    label = json.load(labels)
    label= list(label.values())

path_json = './doc/label_train.json'
with open(path_json) as f:
    label_train = json.load(f)

    

path = './dataset/data_set_minh/train/'

list_dir = os.listdir(path)
print(list_dir)
result = np.empty((0,20), int)


path_name_all_imgs = []
for name_folder in list_dir:
    path_folder = path + name_folder +'/'
    for name in glob.glob(path_folder+'*.jpg'):
        path_name_all_imgs.append(name)


import random
random.shuffle(path_name_all_imgs)

start = time.time()
name_imgs = []
k = 0
if True:
    for name in path_name_all_imgs:
        ids = name.split('/')[-1].split('.')[0]
        
        image_path = name
        ids=name.split('/')[-1].split('.')[0]
        src = load_image(image_path)
        out = model.forward(src)
        pred = torch.gt(out, torch.ones_like(out)/2 )  # threshold=0.5
        pred = pred.tolist()[0]

        for inter, i in enumerate(pred):
            if i:
                pred[inter] = 1
            else:
                pred[inter] = 0
#         print(pred)
#         print(label_train[ids])
#         print('------------------')
#         print(type(pred))
        compare = xnor(label_train[ids],  pred)
        compare = np.array(compare)
        result = np.vstack((result, compare))  
        k+=1
        if k%1000==0:
            stop = time.time()
            t = stop-start
            print(f'{k} :  {t}')
            start = time.time()
#             break

np.save("result_net_50_4.npy", result)
a = 0
for count, i in enumerate(label):
    check = result[:, count]
    number = 0
    total = 0
    for j in check:
        if j == 1:
            number += 1
            total += 1
        elif j == 0:
            total += 1
    try:
        print(i, round(number/total, 4))
        a = a + round(number/total, 4)
    except Exception as e:
        print("An exception occurred: ", e) 
        print(i)
print(' acc = ', a/18)

['taxi', 'car', 'bus', 'lorry', 'truck', 'coach']


/home/minh/anaconda3/envs/tracking/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


1000 :  131.5460398197174
2000 :  130.7918291091919
3000 :  131.48395347595215
4000 :  132.8676266670227
5000 :  132.32146501541138
6000 :  121.79972314834595
7000 :  131.59178972244263
8000 :  133.39595079421997
9000 :  130.71157360076904
10000 :  125.57982850074768
11000 :  124.99978804588318
12000 :  124.76387929916382
13000 :  125.82715272903442
14000 :  122.61256313323975
15000 :  122.49200534820557
16000 :  123.06104397773743
17000 :  123.45031023025513
18000 :  122.7711009979248
19000 :  126.27609825134277
20000 :  127.25250267982483
21000 :  124.68586254119873
22000 :  123.29189085960388
23000 :  129.69568538665771
24000 :  123.76400518417358
25000 :  119.45181274414062
26000 :  129.85173535346985
27000 :  123.84714579582214
28000 :  125.44477701187134
29000 :  126.81108736991882
30000 :  122.00031971931458
31000 :  123.19579696655273
32000 :  127.45515775680542
33000 :  120.95656967163086
34000 :  127.30780100822449
35000 :  125.72385883331299
36000 :  127.38773107528687
37000

In [2]:
a/16

0.797575